In [1]:
#labeling USA border segments
#!/usr/bin/python
import os
from  osgeo import ogr, osr
import pickle
from shapely.geometry import LinearRing,Point,mapping
import numpy as np
import time
import ntpath
import math

#np.set_printoptions(precision=12)
np.set_printoptions(suppress=True)
INF = 999

def euclidian(arr_x1y1x2y2):
    '''
    Compute euclidian distance between points (x1,y1)--(x2,y2) in numpy array.
    '''
    diffs= arr_x1y1x2y2[:,0:2] - arr_x1y1x2y2[0:,2:]
    sqsums = np.sum(diffs**2,axis=1)
    dist = np.sqrt(sqsums)
    dist = dist.reshape(len(dist),1)
    return dist
    
def minxy(x1,y1,x2,y2):
    return min(x1,x2), min(y1,y2)

def maxxy(x1,y1,x2,y2):
    return max(x1,x2), max(y1,y2)
def eqsegment(seg1p1,seg1p2,seg2p1,seg2p2,labeling=False): #tuples of points
    if not labeling:
        s0=sorted([seg1p1,seg1p2])
        s1=sorted([seg2p1,seg2p2])
        if s0[0][0] == s1[0][0] and s0[0][1] == s1[0][1] and s0[1][0] == s1[1][0] and s0[1][1] == s1[1][1]:
            return True
        return False

home =r'../'
srcshpfile = home + r'out/usa.shp'
destdir = home+'out/tmp/'
driver = ogr.GetDriverByName('ESRI Shapefile')
stat ={}
st = time.time()
shp = driver.Open(srcshpfile)
inlayer = shp.GetLayer()
inspref = inlayer.GetSpatialRef()
#outDataSet,outLayer, outLayerDefn = createOutLayer(inlayer,None,"segment-labeled",inlayer.GetSpatialRef(),outgeomtype=ogr.wkbMultiPolygon)
#read the features one by one.
inFeature = inlayer.GetNextFeature()
rings=[]
ringids=[]
sttime=time.time()
while inFeature:
    geom= inFeature.GetGeometryRef()
    if geom.GetGeometryName() == 'POLYGON':
        if geom.GetGeometryCount()>=1:
            for i in xrange(geom.GetGeometryCount()):
                ring= geom.GetGeometryRef(i)
                shapelyRing= LinearRing(ring.GetPoints())
                if not shapelyRing.is_ccw:
                    shapelyRing= LinearRing(reversed(ring.GetPoints()))
                rings+=[shapelyRing]
                try:
                    #print("ringid"),int(inFeature.GetField(0))
                    ringids+=[int(inFeature.GetField(0))]#ringids+=['p'+str(inFeature.GetField(0))+'r'+str(i)]
                except:
                    raise ValueError("Feature ids must be integer.")
    inFeature = inlayer.GetNextFeature()
inFeature=shp=driver=None
#end-while
print("Completed finding all the polygons as rings in the shape file."),len(rings),len(ringids) #list of references to rings, list of ids for the rings.
print("time: {0:.3f}".format(time.time()-st))
outfilename=ntpath.basename(srcshpfile).split(".")[0]

Completed finding all the polygons as rings in the shape file. 1 1
time: 0.319


In [2]:
#collect all the segments in each ring in rings.
if type(rings) != list:
    rings=[rings]
if type(ringids) !=list:
    ringids=[ringids]
if len(rings) != len(ringids):
    raise ValueError("Size of rings and their ids did not match.")

st=time.time()
L = np.zeros((1,5)) #dummy variable for [rid,x1,y1,x2,y2]
for ringid in xrange(len(rings)):
    arr=None
    ring=rings[ringid]
    if not ring.is_ccw:
        raise ValueError("Polygon ring at index %r is not in anti-clockwise order."%ringid)
    arr = np.asarray(list(ring.coords)) #x1,y1,z1
    if ring.has_z:
        arr= arr[:,0:2] #exclude z.
    arr = np.hstack((arr[0:-1,:],arr[1:,:])) #shifting x1,y1,x2,y2
    rids = np.asarray([ringids[ringid]]*arr.shape[0]).reshape((arr.shape[0],1))
    arr = np.hstack((rids,arr))
    L = np.vstack((L,arr))
#end-for
L=L[1:,:] #exclude top row. that are zeros for rid,x1,y1,x2,y2,list of [rid,x1,x2,y1,y2]

stat['og,L.shape']=[L.shape]
stat['og']=[np.unique(L[:,1:2],axis=0).shape,np.unique(L[:,2:3],axis=0).shape]
print("Completed listing segments from all the rings/polygon"), L.shape #rid,x1,y1,x2,y2
print("time: {0:.3f}".format(time.time()-st))
stat

Completed listing segments from all the rings/polygon (56282, 5)
time: 0.331


{'og': [(55153, 1), (55191, 1)], 'og,L.shape': [(56282, 5)]}

In [3]:
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
print pd.DataFrame(L[0:10,0:])

     0          1          2          3          4
0  0.0 -80.661032  24.900218 -80.660666  24.899147
1  0.0 -80.660666  24.899147 -80.660431  24.898460
2  0.0 -80.660431  24.898460 -80.659485  24.898077
3  0.0 -80.659485  24.898077 -80.657695  24.897354
4  0.0 -80.657695  24.897354 -80.656264  24.898582
5  0.0 -80.656264  24.898582 -80.653136  24.901102
6  0.0 -80.653136  24.901102 -80.650015  24.902918
7  0.0 -80.650015  24.902918 -80.646459  24.905191
8  0.0 -80.646459  24.905191 -80.641907  24.908400
9  0.0 -80.641907  24.908400 -80.636338  24.912276


In [4]:
#Transform Geographic Cordinates to a Projected Cordinate System.
epsgdic = {'nad83':4269,'wgs84':4326,'pseudoutm':3857,'worldmercater':3395}
def transform_cordinates(segment_tuple,inspref, outref):
    rid,x1,y1,x2,y2 = segment_tuple
    pointGeom = ogr.Geometry(ogr.wkbPoint)
    pointGeom.AssignSpatialReference(inspref)
    pointGeom.AddPoint(x1,y1)
    pointGeom.TransformTo(outref)
    x1,y1 = pointGeom.GetPoint()[0],pointGeom.GetPoint()[1] #lat,lon
    #
    #pointGeom = ogr.Geometry(ogr.wkbPoint)
    pointGeom.AssignSpatialReference(inspref)
    pointGeom.AddPoint(x2,y2)
    pointGeom.TransformTo(outref)
    x2,y2 = pointGeom.GetPoint()[0],pointGeom.GetPoint()[1]
    return [rid,x1,y1,x2,y2]

outprjref = osr.SpatialReference()
outprjref.ImportFromEPSG(epsgdic["worldmercater"])
st= time.time()
Lprj = np.apply_along_axis(transform_cordinates,1,L,inspref, outprjref) #rid,x1,y1,x2,y2
print("Completed co-ordinate transformation"), Lprj.shape, "target spatial ref", epsgdic["worldmercater"]
print("time: {0:.3f}".format(time.time()-st))
#save Lp in .shp file, save outprj in .prj file
outfilename=outfilename+'.prj'

stat['txn,Lprj.shape']=[Lprj.shape]
stat['txn']=[np.unique(Lprj[:,1:2],axis=0).shape,np.unique(Lprj[:,2:3],axis=0).shape]
print stat
np.unique(np.vstack((Lprj[:,1:2],Lprj[:,3:4]))).shape

Completed co-ordinate transformation (56282, 5) target spatial ref 3395
time: 10.568
{'og': [(55153, 1), (55191, 1)], 'txn,Lprj.shape': [(56282, 5)], 'og,L.shape': [(56282, 5)], 'txn': [(55153, 1), (55191, 1)]}


(55153,)

In [5]:
print pd.DataFrame(Lprj[0:10,0:])

     0             1             2             3             4
0  0.0 -8.979145e+06  2.845509e+06 -8.979104e+06  2.845378e+06
1  0.0 -8.979104e+06  2.845378e+06 -8.979078e+06  2.845295e+06
2  0.0 -8.979078e+06  2.845295e+06 -8.978973e+06  2.845248e+06
3  0.0 -8.978973e+06  2.845248e+06 -8.978774e+06  2.845160e+06
4  0.0 -8.978774e+06  2.845160e+06 -8.978614e+06  2.845309e+06
5  0.0 -8.978614e+06  2.845309e+06 -8.978266e+06  2.845617e+06
6  0.0 -8.978266e+06  2.845617e+06 -8.977919e+06  2.845839e+06
7  0.0 -8.977919e+06  2.845839e+06 -8.977523e+06  2.846116e+06
8  0.0 -8.977523e+06  2.846116e+06 -8.977016e+06  2.846508e+06
9  0.0 -8.977016e+06  2.846508e+06 -8.976396e+06  2.846981e+06


In [6]:
#calculate Maximum/Minimum distance between any two connected points
dist=euclidian(Lprj[0:,1:5])
print dist.shape
print"max dist",dist.max(axis=0),L[dist.argmax(axis=0)]
print "min dist",dist.min(axis=0),L[dist.argmin(axis=0)]

(56282, 1)
max dist [71786.04773421] [[   0.         -112.36503005   48.99884087 -113.009895     48.998619  ]]
min dist [0.05969216] [[   0.         -106.246203     31.541153   -106.24620281   31.54115257]]


In [7]:
#Swap points A(x1,y1) and B(x2,y2) if x1 > x2, mark the swapping.
Lswp=Lprj[0:,:]
st = time.time()
m,n = Lswp.shape
isswapped = np.zeros(m).reshape(m,1)
Lswp = np.hstack((Lswp,isswapped))
def swap_endpoints(c):
    if c[3] < c[1]:
        t1,t2 = c[1],c[2]
        c[1]= c[3];c[2] =c[4];c[3] = t1;c[4] = t2; c[5] = 1
    return c
Lswp = np.apply_along_axis(swap_endpoints,1,Lswp) #arr --> [rid,x1,y1,x2,y2,iswp?]
inf = np.asarray([INF]*m).reshape(m,1) #without dummy first row; right-labels for each rid.
Lswp = np.hstack((Lswp,Lswp[:,0:1],inf)) #arr--> [rid,x1,y1,x2,y2,iswp?,L-labels,R-labels] #left-labels for each rid.
print("Completed end-point swapping"), Lswp.shape,Lswp.dtype

print "min x: {0:,}, y: {1:,}".format(*minxy(*np.min(Lswp[0:,1:5],axis=0)))
print "max x: {0:,}, y: {1:,}".format(*maxxy(*np.max(Lswp[0:,1:5],axis=0)))

print("time: {0:.3f}".format(time.time()-st))
outfilename=outfilename

stat['2swp,Lswp.shape']=[Lswp.shape]
stat['2swp']=[np.unique(Lswp[:,1:2]).shape,np.unique(Lswp[:,2:3]).shape]
print stat
np.unique(np.vstack((Lswp[:,1:2],Lswp[:,3:4]))).shape

Completed end-point swapping (56282, 8) float64
min x: -13,885,233.4147, y: 2,845,159.55756
max x: -7,452,828.22006, y: 6,307,879.04091
time: 0.181
{'txn': [(55153, 1), (55191, 1)], 'og': [(55153, 1), (55191, 1)], 'txn,Lprj.shape': [(56282, 5)], '2swp,Lswp.shape': [(56282, 8)], '2swp': [(51190,), (51127,)], 'og,L.shape': [(56282, 5)]}


(55153,)

In [8]:
print pd.DataFrame(Lswp[0:10,0:])

     0             1             2             3             4    5    6      7
0  0.0 -8.979145e+06  2.845509e+06 -8.979104e+06  2.845378e+06  0.0  0.0  999.0
1  0.0 -8.979104e+06  2.845378e+06 -8.979078e+06  2.845295e+06  0.0  0.0  999.0
2  0.0 -8.979078e+06  2.845295e+06 -8.978973e+06  2.845248e+06  0.0  0.0  999.0
3  0.0 -8.978973e+06  2.845248e+06 -8.978774e+06  2.845160e+06  0.0  0.0  999.0
4  0.0 -8.978774e+06  2.845160e+06 -8.978614e+06  2.845309e+06  0.0  0.0  999.0
5  0.0 -8.978614e+06  2.845309e+06 -8.978266e+06  2.845617e+06  0.0  0.0  999.0
6  0.0 -8.978266e+06  2.845617e+06 -8.977919e+06  2.845839e+06  0.0  0.0  999.0
7  0.0 -8.977919e+06  2.845839e+06 -8.977523e+06  2.846116e+06  0.0  0.0  999.0
8  0.0 -8.977523e+06  2.846116e+06 -8.977016e+06  2.846508e+06  0.0  0.0  999.0
9  0.0 -8.977016e+06  2.846508e+06 -8.976396e+06  2.846981e+06  0.0  0.0  999.0


In [9]:
stat={}
Lswp.shape

(56282, 8)

In [10]:
#Given end-points A(x1,x2), B(x2,y2) with origin A, dest B, (if is swapped=1, then origin is B, dest A.)
print("Top/Bottom labeling started; got segment-tuples of size "), Lswp.shape, type(Lswp)
cnt=0
st = time.time()
idx = np.array([i for i in range(Lswp.shape[0])]).reshape(Lswp.shape[0],1)
L= np.hstack((idx,Lswp))
print pd.DataFrame(L[0:10,0:])
L = L[L[:,2].argsort()] #sort by x1.;#L= sorted(L, key=lambda t: t[1]) 
print pd.DataFrame(L[0:10,0:])
#append isvisited? column
idx=L[:,0:1] #initial idx.
L=L[:,1:]
##
isvisited = np.zeros((len(L),1))
L = np.hstack((L,isvisited)) #L's columns--> [rid,x1,y1,x2,y2,iswp?,rid-left,rid-right,isvisited?]
for ti in range(len(L)): #each tuple-index
    t = L[ti] #[rid,x1,y1,x2,y2,iswp?,rid-left,rid-right,isvisited?]
    polyid,s1x1,s1y1,s1x2,s1y2,s1isswapped = t[0:6]
    #check is swapped.
    if s1isswapped:x1,y1,x2,y2 = s1x2,s1y2,s1x1,s1y1 #get original arrangement of points for the seg.
    else:x1,y1,x2,y2 = s1x1,s1y1,s1x2,s1y2
    if t[-1] == False: #not visited
        for nti in range(ti+1,len(L)):#new tuple index
            newt = L[nti]
            polyid_p,s2x1,s2y1,s2x2,s2y2,s2isswapped = newt[0:6]
            if s1x1 < s2x1: #no matching segment found
                break
            if eqsegment((s1x1,s1y1),(s1x2,s1y2),(s2x1,s2y1),(s2x2,s2y2)):
                cnt+=1
                #visited !!
                t[-1]=True
                newt[-1]=True #visited
                #now do casw-wise.
                if x1 < x2: #not swapped
                    t[6] = polyid #top
                    t[7] = polyid_p #bot
                    newt[6] =  polyid#same code.
                    newt[7] = polyid_p
                    break
                elif x1 > x2:
                    t[6] = polyid_p#top
                    t[7] = polyid#bot
                    newt[6] = polyid_p
                    newt[7] = polyid
                    break
                elif x1 == x2:
                    if y1 < y2: #vup
                        t[6] = polyid #top
                        t[7] = polyid_p #bot
                        newt[6] = polyid_p
                        newt[7] = polyid  
                        break
                    elif y1 > y2: #vdown
                        t[6] = polyid_p #top
                        t[7] = polyid #bot
                        newt[6] = polyid_p  
                        newt[7] = polyid   
                        break  #exit form current loop.
        #end-for
        if t[-1] == False: #this segment is no shared by any other rings.
            t[-1]=True #now visti
            if x1 < x2:
                t[6] = polyid #top
                t[7] = INF #bot
            elif x1 > x2:
                t[6] = INF #top
                t[7] = polyid #bot
            elif x1 == x2:
                if y1 < y2: #vup
                    t[6] = polyid #top
                    t[7] = INF #bot
                elif y1 > y2: #vdown
                    t[6] = INF #top
                    t[7] = polyid #bot
        #end-if
    #end-if
L = L[0:,0:-1]#exclude isvisited col.
L = np.hstack((idx,L))
L = L[L[:,0].argsort()] #sort by index again.
L = L[0:,1:] #exclude idx.
#sL = L[L[:,0].argsort()] #sort by state-id <rid,x1,y1,x2,y2,isswp,topl,botl> #whis does not order the points.
sL = L
print "Completed labeling top/bottom for the segments.", sL.shape
print("time: {0:.3f}".format(time.time()-st))
outfilename=outfilename+'.lbl'

stat['lbl,Lprj.shape']=[sL.shape]
stat['lbl']=[np.unique(sL[:,1:2],axis=0).shape,np.unique(sL[:,2:3],axis=0).shape]
stat

Top/Bottom labeling started; got segment-tuples of size  (56282, 8) <type 'numpy.ndarray'>
     0    1             2             3             4             5    6    7      8
0  0.0  0.0 -8.979145e+06  2.845509e+06 -8.979104e+06  2.845378e+06  0.0  0.0  999.0
1  1.0  0.0 -8.979104e+06  2.845378e+06 -8.979078e+06  2.845295e+06  0.0  0.0  999.0
2  2.0  0.0 -8.979078e+06  2.845295e+06 -8.978973e+06  2.845248e+06  0.0  0.0  999.0
3  3.0  0.0 -8.978973e+06  2.845248e+06 -8.978774e+06  2.845160e+06  0.0  0.0  999.0
4  4.0  0.0 -8.978774e+06  2.845160e+06 -8.978614e+06  2.845309e+06  0.0  0.0  999.0
5  5.0  0.0 -8.978614e+06  2.845309e+06 -8.978266e+06  2.845617e+06  0.0  0.0  999.0
6  6.0  0.0 -8.978266e+06  2.845617e+06 -8.977919e+06  2.845839e+06  0.0  0.0  999.0
7  7.0  0.0 -8.977919e+06  2.845839e+06 -8.977523e+06  2.846116e+06  0.0  0.0  999.0
8  8.0  0.0 -8.977523e+06  2.846116e+06 -8.977016e+06  2.846508e+06  0.0  0.0  999.0
9  9.0  0.0 -8.977016e+06  2.846508e+06 -8.976396e+06  2.84

{'lbl': [(51190, 1), (51127, 1)], 'lbl,Lprj.shape': [(56282, 8)]}

In [11]:
print pd.DataFrame(L[0:10,0:])

     0             1             2             3             4    5    6      7
0  0.0 -8.979145e+06  2.845509e+06 -8.979104e+06  2.845378e+06  0.0  0.0  999.0
1  0.0 -8.979104e+06  2.845378e+06 -8.979078e+06  2.845295e+06  0.0  0.0  999.0
2  0.0 -8.979078e+06  2.845295e+06 -8.978973e+06  2.845248e+06  0.0  0.0  999.0
3  0.0 -8.978973e+06  2.845248e+06 -8.978774e+06  2.845160e+06  0.0  0.0  999.0
4  0.0 -8.978774e+06  2.845160e+06 -8.978614e+06  2.845309e+06  0.0  0.0  999.0
5  0.0 -8.978614e+06  2.845309e+06 -8.978266e+06  2.845617e+06  0.0  0.0  999.0
6  0.0 -8.978266e+06  2.845617e+06 -8.977919e+06  2.845839e+06  0.0  0.0  999.0
7  0.0 -8.977919e+06  2.845839e+06 -8.977523e+06  2.846116e+06  0.0  0.0  999.0
8  0.0 -8.977523e+06  2.846116e+06 -8.977016e+06  2.846508e+06  0.0  0.0  999.0
9  0.0 -8.977016e+06  2.846508e+06 -8.976396e+06  2.846981e+06  0.0  0.0  999.0


In [12]:
#Translate to first quadrant
import matrices as MATRIX
reload(MATRIX)
try:
    TVECTOR = pickle.load(open(home+r'out/translation_vector.p','rb'))
except:
    TVECTOR = pickle.load(open(home+r'out/translation_vectorwin.p','rb')) #supported by windows.

st = time.time()
TM = MATRIX.translationMatrix(-TVECTOR['xoff'],-TVECTOR['yoff'],0,ndim=2)
m,n = sL[0:, 1:5].shape

#translation of first end-points
Lp1 = sL[0:,1:3] #first end-points
ones = np.ones(m).reshape(m,1)
Lp1 = np.hstack((Lp1,ones))
TLp1 = np.dot(Lp1,TM) #exclude ?iswap column.
n=Lp1 = None

#translation of second end-points
Lp2= sL[0:,3:5] #second end-points
Lp2 = np.hstack((Lp2,ones)) 
TLp2 = np.hstack((Lp2,ones))
TLp2 = np.dot(Lp2, TM)[0:]
m=ones=Lp2 = None

TL = np.hstack((sL[0:,0:1],TLp1[:,:-1],TLp2[:,:-1],sL[:,5:])) #rid,x1,y1,x2,y2,swp?Top,Bot
sL=Tp1=Tp2=None

#outfilename=outfilename+'.txn'
print("Completed Translation to first quadrant."), TL.shape
print "min x,y",minxy(*np.min(TL[0:,1:5],axis=0))
print "max x,y",maxxy(*np.max(TL[0:,1:5],axis=0))
print("time: {0:.3f}".format(time.time()-st))
outfilename=outfilename+'.txn'

Completed Translation to first quadrant. (56282, 8)
min x,y (6152274.928091211, 18341730.29726051)
max x,y (12584680.122736868, 21804449.780606803)
time: 0.022


In [13]:
#Round and Make x1,y1,x2, y1 into Integers
import sys
np.set_printoptions(precision=6)
np.set_printoptions(suppress=True)

mfactor=100#multiply x1,x2,y1,y2 by 100 and then round.
TL1 = np.hstack((TL[0:,0:1], np.round(TL[0:,1:5]*mfactor,0), TL[0:, 5:]))#rid, r
#round(x1,y1,x2,y2),isswp?,Top,Bot

print("Completed factoring by 100 and rounding to int."),TL1.shape

print "min x: {0:,.0f}, y:{1:,}".format(*minxy(*np.min(TL1[0:,1:5],axis=0)))
print "max x: {0:,.0f}, y:{1:,}".format(*maxxy(*np.max(TL1[0:,1:5],axis=0))) +"\n"
print "permissible 32 bit x,y? Ans:",4294967295 > max(maxxy(*np.max(TL1[0:,1:5],axis=0)))
outfilename=outfilename+'.int'

Completed factoring by 100 and rounding to int. (56282, 8)
min x: 615,227,493, y:1,834,173,030.0
max x: 1,258,468,012, y:2,180,444,978.0

permissible 32 bit x,y? Ans: True


In [18]:
#Annotate polygion-ids by polygon-names like state-names. 
def annotate_top_bot(tup,SC):
    try:
        tup[0] = SC[int(tup[0])].replace(" ", "")[0:10]
    except:
        pass
    try:
        tup[-1] = SC[int(tup[-1])].replace(" ", "")[0:10]
    except:
        pass
    try:
        tup[-2] =SC[int(tup[-2])].replace(" ", "")[0:10]
    except Exception, e:
        pass
    return tup

try:
    SC = pickle.load(open(home+'out/state_dic.p','rb'))
except:
    SC = pickle.load(open(home+'out/state_dicwin.p','rb'))

arr=TL1[0:,0:]
arr = arr.astype(object)
print arr.shape
np.apply_along_axis(annotate_top_bot,1,arr,SC)
print arr.shape
if not os.path.exists(destdir):
    os.makedirs(destdir)
np.savetxt(destdir+outfilename+'.txt',arr[0:,0:],fmt='%d '+ '%010d '*4+'%0d '+'%d '+'%d',header="state intx1 inty1 intx2 inty2 isswp top bot")
print "data saved in {0}".format(destdir+outfilename+'.txt')

(56282, 8)
(56282, 8)
data saved in ../out/tmp/usa.prj.lbl.txn.int.txt


In [15]:
#sanity check by ploting the segments.

In [16]:
'''
#plot and see if a map is correct.

import matplotlib.pyplot as plt
import numpy as np
import matplotlib

i,j = np.where(TL1[0:,0:1] == 18)
selpoints = TL1[(i,j),1:3][0]/1000
ring = selpoints.tolist()
print ring[0], ring[-1]
ring = ring+ ring[0:1]
print ring[0], ring[-1]
ring =ring 
# Fixing random state for reproducibility
x=[t[0] for t in ring]
y=[t[1] for t in ring]
plt.plot(x, y,c="g", alpha=0.5, marker='.',
            label="Luck")
plt.xlabel("Leprechauns")
plt.ylabel("Gold")
plt.legend(loc=2)
plt.show()
from shapely.geometry import LinearRing,Point,mapping
shring1 = LinearRing(ring)
shring1
'''

'\n#plot and see if a map is correct.\n\nimport matplotlib.pyplot as plt\nimport numpy as np\nimport matplotlib\n\ni,j = np.where(TL1[0:,0:1] == 18)\nselpoints = TL1[(i,j),1:3][0]/1000\nring = selpoints.tolist()\nprint ring[0], ring[-1]\nring = ring+ ring[0:1]\nprint ring[0], ring[-1]\nring =ring \n# Fixing random state for reproducibility\nx=[t[0] for t in ring]\ny=[t[1] for t in ring]\nplt.plot(x, y,c="g", alpha=0.5, marker=\'.\',\n            label="Luck")\nplt.xlabel("Leprechauns")\nplt.ylabel("Gold")\nplt.legend(loc=2)\nplt.show()\nfrom shapely.geometry import LinearRing,Point,mapping\nshring1 = LinearRing(ring)\nshring1\n'

In [17]:
from shapely.geometry import LinearRing,Point,mapping
print np.unique(TL1[0:,0:1])
#load initial vertex order files.
#save to shp.
outputShapefile=destdir+outfilename+'.shp'
driver = ogr.GetDriverByName('ESRI Shapefile')
outprjref = osr.SpatialReference()
outprjref.ImportFromEPSG(epsgdic["worldmercater"])

if os.path.exists(outputShapefile):
    driver.DeleteDataSource(outputShapefile)
outDataSet = driver.CreateDataSource(outputShapefile)
outLayer = outDataSet.CreateLayer("mystates", outprjref, geom_type=ogr.wkbMultiPolygon)
outLayer.CreateField(ogr.FieldDefn('STATEFP'),ogr.OFTInteger)
outLayerDefn = outLayer.GetLayerDefn()

##assume that the boundary points are in order in the array.
for polyid in np.unique(TL1[0:,0:1]):
    i,j = np.where(TL1[0:,0:1] == polyid)
    selpoints = TL1[(i,j),1:6][0] #
    
    poly = ogr.Geometry(ogr.wkbPolygon)
    ring = ogr.Geometry(ogr.wkbLinearRing)
    
    def original_dirn(segment_tup):
        x1,y1,x2,y2,iswp=segment_tup
        if int(iswp):
            ring.AddPoint(x2,y2)
            return []
        ring.AddPoint(x1,y1)
        return []
        
    selpoints = np.apply_along_axis(original_dirn,1,selpoints)
    pts = selpoints.tolist()
    #pts +=pts + pts[0:1] #completing list.
    poly.AddGeometry(ring)

    outFeature = ogr.Feature(outLayerDefn)
    outFeature.SetGeometry(poly)
    outFeature.SetField(outLayerDefn.GetFieldDefn(0).GetNameRef(), polyid)
    outLayer.CreateFeature(outFeature)

#end-for
outLayer=outFeature=poly=plist=ring=outLayerDefn=outLayer=outDataSet=None
print outputShapefile

[0.]
../out/tmp/usa.prj.lbl.txn.int.shp
